In [1]:
from resume import my_resume

In [1]:
from IPython.display import display, HTML, Markdown, clear_output, FileLink
from jobs_skills_weights import get_jobs, get_raw_job_details
import skill_weights
import os, shutil, re
from mark_status import mark_status
import requests

In [6]:
my_resume.write_html_to_file(
    stylesheet='chalkboard',
    filepath="docs/index.html",
    should_render_all=True
)

In [7]:
my_resume.write_html_to_file(
    stylesheet='lighttheme',
    filepath="docs/resumes/index.html",
    should_render_all=True
)

In [25]:
my_resume.export_pdf(
    "docs/resumes/full.pdf",
    alt_template_prefixes={"*":"pdf"},
    should_render_all=True,
    goal_small=True
)

# Pulling Job Data

In [10]:
jobs = get_jobs()
raw_job_details = get_raw_job_details(jobs)
# job_descriptions = skill_weights.get_job_descriptions(raw_job_details)
job_skill_weights = skill_weights.get_job_skill_weights(raw_job_details)

NameError: name 'jobs' is not defined

# Generation

In [12]:
skill_text_weights_con = lambda weights, job_id: weights.loc[job_id, :].groupby(level=0).apply(lambda df: df.xs(df.name).to_dict('index')).to_dict()

In [ ]:
# assert input("Ready to generate? Type 'OK'") == "OK"
# for job_id, job_data in jobs.iterrows():
#     private_path = f"docs/pdf_sources/{job_id}"
#     os.makedirs(private_path, exist_ok=True)
#     my_resume.write_html_to_file(
#         stylesheet='lighttheme',
#         filepath=f"{private_path}/Zach_Allen_Resume.html",
#         alt_template_prefixes={'*':'pdf'},
#         skill_text_weights = skill_text_weights_con(job_skill_weights, job_id)
#     )

In [ ]:
pdf_upload_path = "docs/pdf_sources/upload/Zach_Allen_Resume.pdf"
os.makedirs("docs/pdf_sources/upload", exist_ok=True)

In [ ]:
# global_skill_text_weights = job_skill_weights.groupby(level=["id", "skill", "skill text"]
#         ).sum().groupby(level=["skill", "skill text"]
#         ).sum().divide(len(jobs)).groupby(level=0
#         ).apply(lambda df: df.xs(df.name).to_dict('index')).to_dict()

In [ ]:
for job_id, job_data in jobs.sort_index().iterrows():
    clear_output()
    
    # Display links to job info
    print(job_id)
    print(job_data['company_name'])
    display(HTML(f"<a href='{job_data['url']}'>job</a>"))
    
    # Construct the PDF
    os.makedirs(f"docs/pdf_sources/{job_id}", exist_ok=True)
    pdf_fpath = f"docs/pdf_sources/{job_id}/Zach_Allen_Resume.pdf"
    my_resume.export_fitted_pdf(
        pdf_fpath,
        
        highest_threshold=0.05,
        lowest_threshold=-1e-3,
        skill_cost=0.1,
        should_render_all_skills=True,
        
        stylesheet='lighttheme',
        alt_template_prefixes={'*':'pdf'},
        skill_text_weights = skill_text_weights_con(job_skill_weights, job_id)
    )
    
    # Copy the output PDF to a single location for easy upload
    try:
        shutil.copyfile(pdf_fpath, pdf_upload_path);
    except shutil.SameFileError: # Sometimes this comes up when re-running a job
        pass
    
    # Wait for feedback on submission status
    status = input("Status:")
    if status == "skip":
        continue;
    if status == "":
        status = "applied"
    
    mark_status(job_id, status_key_or_id = status)
    